In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import sys, os
pbin_dir = os.path.dirname(sys.executable)
%env PBIN_DIR={pbin_dir}
os.environ['PATH'] = f'{os.environ["PBIN_DIR"]}:{os.environ["PATH"]}'
print(os.environ['PATH'])

env: PBIN_DIR=/share_6/work/solepro-moriya/jupyter102/venv-3.9.10-cikm/bin
/share_6/work/solepro-moriya/jupyter102/venv-3.9.10-cikm/bin:/root/.pyenv/versions/3.9.10/bin:/root/.pyenv/libexec:/root/.pyenv/plugins/python-build/bin:/root/.pyenv/shims:/root/.pyenv/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin


In [3]:
#!${PBIN_DIR}/pip install pydash
import fileinput, json, copy, pydash, random
from tqdm.auto import tqdm
def randomize_cands(cands,true_index):
    inds = list(range(len(cands)))
    inds=random.sample(inds,len(inds))
    cands=[cands[i] for i in inds]
    label=inds.index(true_index)
    return [cands,label]
def add_response_cands(all_uttrs,cands,count,exclude_responses):
    l=len(cands)+count
    while True:
        if len(cands) == l:
            break
        u = random.choice(all_uttrs)
        if u in cands:
            continue
        if u in exclude_responses:
            continue
        cands.append(u)
    return cands
def get_tsv_groups(div,grouplen):
    with open(f'/share_7/projects/hais/data/msc_bertfp/last_history/{div}.tsv',encoding='utf-8') as f:
        lines=[x.strip().split('\t') for x in f.read().strip().split('\n')]
    groups=[]
    group = None
    for line in lines:
        if line[0] == '1':
            if group is not None:
                assert(len(group)==grouplen)
                groups.append(group)
            group = []
        session,author=line[1].split(' ',maxsplit=1)
        group.append([session,author,line[-1].strip()])
    assert(len(group)==grouplen)
    groups.append(group)
    return groups
def create_data(div,tsvdiv,tsvgrouplen,respcount,outfile):
    convs = {}
    prev_count = 0
    settings=[
        [f'/share_6/work/solepro-moriya/jupyter102/selfplay/msc/msc_dialogue/session_2/{div}.txt', True],
        [f'/share_6/work/solepro-moriya/jupyter102/selfplay/msc/msc_dialogue/session_3/{div}.txt', False],
        [f'/share_6/work/solepro-moriya/jupyter102/selfplay/msc/msc_dialogue/session_4/{div}.txt', False]
    ]
    if div != 'train':
        settings.append([f'/share_6/work/solepro-moriya/jupyter102/selfplay/msc/msc_dialogue/session_5/{div}.txt',False])
    all_uttrs=[]
    if True:
        for inp, prev in settings:
            fi = fileinput.FileInput((inp,))
            for line in fi:
                d = json.loads(line)
                ses=d['dialog'][0]['convai2_id']
                if ' ' in ses:
                    assert(False)
                if prev:
                    t = [[x['text'].strip(),None] for x in d['previous_dialogs'][0]['dialog']] 
                    convs[ses] = {
                        'u':t
                    }
                else:
                    if 'i' not in convs[ses]:
                        #index0のidがおかしい場合があるので避けて取る
                        convs[ses]['i']=[x['id'] for x in d['dialog']][2:4]
                        #if convs[ses]['i'][0]=='56152':
                        #    print(inp)
                        #    print(ses)
                        #    print(d['dialog'][0])
                        #    print(convs[ses])
                        convs[ses]['p']=d['personas']
                convs[ses]['u'] += [[x['text'].strip(),x['id']] for x in d['dialog']]
    for ses in convs:
        all_uttrs+=[x[0] for x in convs[ses]['u']]
        all_uttrs+=pydash.flatten(convs[ses]['p'])
    all_uttrs=pydash.uniq(all_uttrs)
    return all_uttrs
all_uttrs=create_data('train','',10,10,'a')
#!${PBIN_DIR}/pip install gensim
import json
import pydash
from torchtext.data import get_tokenizer
from gensim.models import word2vec
tokenizer = get_tokenizer("basic_english")
wordss = [tokenizer(sentence) for sentence in all_uttrs]
#padlen = 50
#for i, words in enumerate(wordss):
#    wordss[i] = ['<PAD>'] * (padlen - len(words)) + words
print(wordss[0])


['i', 'need', 'some', 'advice', 'on', 'where', 'to', 'go', 'on', 'vacation', ',', 'have', 'you', 'been', 'anywhere', 'lately', '?']


In [4]:
import multiprocessing
#https://radimrehurek.com/gensim/models/word2vec.html#
#https://rare-technologies.com/word2vec-tutorial/
model = word2vec.Word2Vec(wordss,vector_size=200,workers=multiprocessing.cpu_count())


In [5]:
wv = model.wv
vocab = {'<PAD>':0}
#PADは本家でも0になってる
embeddings = [[0.0]*200]
print(len(wv))
for i in range(len(wv)):
    vocab[wv.index_to_key[i]] = i+1
    #normalizeして出力する方法もあるがcikmのembeddingみるとしてなさそう
    embeddings.append(wv[i])
import pickle
with open("vocab_and_embeddings_msc.pkl", mode="wb") as f:
    pickle.dump([vocab,embeddings], f)


12527
